## Low Rank Approximation With SVD

### Read in our necessary libraries.

In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np

import pandas as pd
import scipy as sci 
from scipy import linalg

np.set_printoptions(precision = 5, suppress = True, linewidth = 100)

### Read in our MovieLens data 
Note that these data sets are smaller than the actual MovieLens datasets.

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


Notice the end of the movies dataframe there are gaps in the movie id's compared to the index. So we are going to redefine the movie id so that we give the movie id and index based on where it occurs in the dataframe. We are going to be creating a sparse matrix but we are only going to store the non-zero values. The problem is that if we do it with the movie id's we have now, then we will have 193609 columns with most of them consisting of just zeroes. So to save memory, we are going to create a dictionary of key-value pairs for the original movie id and the index.

In [7]:
ReMovieId = dict()
for idx in movies.index:
    ReMovieId[movies.movieId[idx]] = idx
ratings['ReId'] = [ ReMovieId[mId] for mId in ratings.movieId]
ratings.head()

,userId,movieId,rating,ReId
0,1,1,4.0,0
1,1,3,4.0,2
2,1,6,4.0,5
3,1,47,5.0,43
4,1,50,5.0,46


In [9]:
ratings.describe()

,userId,movieId,rating,ReId
count,100836.000000,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557,3107.376235
std,182.618491,35530.987199,1.042529,2633.512801
min,1.000000,1.000000,0.500000,0.000000
25%,177.000000,1199.000000,3.000000,901.000000
50%,325.000000,2991.000000,3.500000,2254.000000
75%,477.000000,8122.000000,4.000000,5105.250000
max,610.000000,193609.000000,5.000000,9741.000000


### Create a sparse matrix of the user ratings
We are going to fill the sparse matrix with the ratings and where that rating occured by using the 'ReId' so that we don't have the large number of zero columns. In other words, for example, say the 5th movie in our data set is actually the movie id '7', then the 5th column of the matrix will be for movie id '7' rather than 5 since we don't have a movie id of '5' or '6'. 

In [10]:
from scipy import sparse

In [11]:
ratingsSparse = sparse.csr_matrix((ratings.rating, (ratings.userId, ratings.ReId)))
ratingsSparse

<611x9742 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

So this matrix is 611 rows by 9742 columns with 100836 non-zero entries. Note that this matrix has 5952362 indices so this matrix about 98% sparse. If we had used movie id's ranging from 1 to 193609, then the aprsity of the matrix would be virtually 100%. Now we can take this sparse matrix and turn it into a sparse dataframe where the zeros are not stored.

In [15]:
ratingsDf = pd.SparseDataFrame(ratingsSparse, columns = movies.title, default_fill_value = 0, dtype = np.int32)
ratingsDf.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the ratings dataframe above, we are seeing the movie rating by user. So in the second row (user 2) we can that they rated 'Toy Story', 'Grumpier Old Men', and 'Heat' a 4. 

In [ ]:
# ratingsDf.to_csv('movielens_small.csv') can export for later if need be.

Now we are going to make this a non-sparse matrix. We can do this because this is a relatively small data set compared to the actual MovieLens data.

In [16]:
Rtngs = ratingsDf.to_dense().values

Well, how much memory would we be using if we stored all of the values?

In [20]:
Rtngs.size*Rtngs.itemsize #number of bytes to store this matrix ~ 46MB

47618896

How much memory would we be using if we used the entire MovieLens data set?

In [22]:
users, movies = 1,1
Rtngs.itemsize  * 280_000 * users * 58_000 * movies  /1024  /1024

123901.3671875

This is about 123GB to store on the computer, which is why we can't do this with the full data set (at least on a laptop). For now this ok becasue we just devloping an algorithm.

### Develop the Algorithm
We are going to start out with an 8 by 7 matrix, but we are going to get it by multiplying two matrices that are rank and corank 3. So the rank of A is going to be 3. I would like to get the SVD of A by making it smaller, in particular, a reduced A matrix with 3 columns. The example is as follows.

In [23]:
B = np.matrix([[1,0,3],
               [4,2,0],
               [0,1,0],
               [1,2,0],
               [0,0,3],
               [0,1,1],
               [1,3,1],
               [1,1,2]]) 

F = np.matrix( [[1,0,2,3,0,2,1],
                [3,1,2,0,1,2,0],
                [1,1,0,3,3,0,1]])

A = B @ F
A

matrix([[ 4,  3,  2, 12,  9,  2,  4],
        [10,  2, 12, 12,  2, 12,  4],
        [ 3,  1,  2,  0,  1,  2,  0],
        [ 7,  2,  6,  3,  2,  6,  1],
        [ 3,  3,  0,  9,  9,  0,  3],
        [ 4,  2,  2,  3,  4,  2,  1],
        [11,  4,  8,  6,  6,  8,  2],
        [ 6,  3,  4,  9,  7,  4,  3]])

In [24]:
A.shape

(8, 7)

### Taking the SVD of A
Notice that the last 4 singular values are zero and the first 3 are non-zero.

In [25]:
U, Sigma, Vt = linalg.svd(A)
Sigma

array([38.85434, 15.29954,  6.65313,  0.     ,  0.     ,  0.     ,  0.     ])

So I can just take the first 3 columns of U and the first 3 columns of V to recover the original matrix.

In [26]:
( U[:,:3] @ np.diag( Sigma[:3]) @ Vt[:3,:] ).astype(int)

array([[ 4,  3,  2, 12,  9,  2,  4],
       [10,  2, 12, 12,  2, 12,  4],
       [ 3,  0,  2,  0,  1,  2,  0],
       [ 7,  2,  6,  3,  2,  6,  1],
       [ 3,  2,  0,  9,  9,  0,  3],
       [ 4,  1,  2,  3,  4,  2,  1],
       [11,  3,  8,  6,  6,  8,  2],
       [ 6,  2,  4,  9,  7,  4,  3]])

So we can obtain the following representation:
# $$\textbf{A}_{8x7} = \textbf{U}_{8x3} \Sigma_{3x3} \textbf{V}^{T}_{3x7}$$

However, $\textbf{U}$ is an 8 x 8 matrix when we do the SVD.

In [27]:
U

array([[-0.36985, -0.52882, -0.21429,  0.71271, -0.06635, -0.13826,  0.07312,  0.02859],
       [-0.57233,  0.43265, -0.63184, -0.14406, -0.12394,  0.09927, -0.06413, -0.18958],
       [-0.08937,  0.12345,  0.27611,  0.00267, -0.68226, -0.01048,  0.64278, -0.14758],
       [-0.27714,  0.29334,  0.2562 ,  0.00374, -0.08475, -0.75424, -0.21903,  0.38399],
       [-0.27145, -0.57526,  0.08172, -0.57334, -0.03944, -0.2762 , -0.10151, -0.41453],
       [-0.17986, -0.0683 ,  0.30335,  0.04763, -0.5    ,  0.44334, -0.63942,  0.11525],
       [-0.457  ,  0.22504,  0.55954,  0.18528,  0.47639,  0.19083,  0.08027, -0.35112],
       [-0.36874, -0.21362,  0.03457, -0.3255 ,  0.17048,  0.3046 ,  0.32217,  0.69686]])

So how are we going to reduce the size? We are going to generate a random matrix $\Omega$ that is linearly independent where,
# $$\Omega = [\omega_{1} | \omega_{2} | \omega_{3}]$$
So these vetors are linearly independent and not in the kernal of $\textbf{A}$. This means if we apply $\textbf{A}$ to $\Omega$, we will only get things in the range of $\textbf{A}$ with 100% proability. Here, we will let,
# $$ \textbf{Y}_{8x3} = \textbf{A}_{8x7} \Omega_{7x3} $$

In [28]:
Ω = np.random.normal(size = (A.shape[1],3) )
Y = A @ Ω
Y

matrix([[ -4.30479,  26.6944 ,  -9.58274],
        [-37.61447,  34.35036,  -1.49662],
        [ -7.76469,   2.10027,  -1.56271],
        [-21.05065,  11.738  ,  -2.71822],
        [  1.21649,  19.15695,  -9.98994],
        [ -7.35919,   8.48592,  -4.89269],
        [-28.40984,  20.22392,  -7.61091],
        [-12.47497,  22.40903,  -7.81547]])

Note that the columns of $\textbf{Y} = [\textbf{y}_{1} |\textbf{y}_{2}|\textbf{y}_{3}]$ are linearly independent and form a column basis for the range of $\textbf{A}$ since $\textbf{A}$ has dimension 3. Now we are going to apply the QR algorithm, where Q is the Gram-Schmidt process applied to the three columns of $\textbf{Y}$.

In [29]:
Q, R = np.linalg.qr(Y, mode = 'reduced') # Q is the Gram-Schmidt process 
                    # applied to Y

In [30]:
Q

matrix([[-0.07919,  0.67386,  0.04482],
        [-0.69193,  0.05047,  0.65777],
        [-0.14283, -0.13605, -0.24607],
        [-0.38723, -0.19146, -0.20466],
        [ 0.02238,  0.59322, -0.24266],
        [-0.13537,  0.06169, -0.32696],
        [-0.52261, -0.12978, -0.53162],
        [-0.22948,  0.34007, -0.12037]])

Notice that $\textbf{q }_{1}^{T} \textbf{q}_{2} = 0$. Which implies the columns of $\textbf{Q}$ are an Orthonormal Basis for the Range of $\textbf{Y} = \textbf{A}\Omega$.

In [32]:
Q[:,0].T @ Q[:,1]

matrix([[-0.]])

Note that since the columns of $\textbf{Q}$ are and Orthonormal Basis, then $\textbf{Q}$ is an orthogonal matrix which implies that $\textbf{Q}\textbf{Q}^{T} = \textbf{I}$. So we have,
# $$\textbf{A} = \textbf{Q}\textbf{Q}^{T}\textbf{A}$$

Let
# $$ \textbf{B} = \textbf{Q}^{T}\textbf{A} \implies \textbf{A} = \textbf{B} \textbf{Q} $$

In [33]:
B = Q.T @ A
B

matrix([[-17.97516,  -5.52122, -16.44017, -15.82084,  -8.09598, -16.44017,  -5.27361],
        [  4.09096,   4.00765,   0.97787,  15.92348,  12.83418,   0.97787,   5.30783],
        [ -4.01968,  -3.07487,   0.87445,   0.37924,  -6.46056,   0.87445,   0.12641]])

Now taking the SVD of $\textbf{B}$, we can see that the singular values of $\textbf{B}$ are the nonzero singular values of $\textbf{A}$

In [34]:
Ur, Sigmar, Vrt = linalg.svd(B, full_matrices = False)
Sigmar, Sigma

(array([38.85434, 15.29954,  6.65313]),
 array([38.85434, 15.29954,  6.65313,  0.     ,  0.     ,  0.     ,  0.     ]))

Then multiplying $\textbf{Q}\textbf{U}_{B}$ (U from the SVD of B), we obtain the first 3 columns of $\textbf{U}$ from the SVD of $\textbf{A}$.

In [35]:
Q @ Ur

matrix([[ 0.36985, -0.52882,  0.21429],
        [ 0.57233,  0.43265,  0.63184],
        [ 0.08937,  0.12345, -0.27611],
        [ 0.27714,  0.29334, -0.2562 ],
        [ 0.27145, -0.57526, -0.08172],
        [ 0.17986, -0.0683 , -0.30335],
        [ 0.457  ,  0.22504, -0.55954],
        [ 0.36874, -0.21362, -0.03457]])

In [36]:
U

array([[-0.36985, -0.52882, -0.21429,  0.71271, -0.06635, -0.13826,  0.07312,  0.02859],
       [-0.57233,  0.43265, -0.63184, -0.14406, -0.12394,  0.09927, -0.06413, -0.18958],
       [-0.08937,  0.12345,  0.27611,  0.00267, -0.68226, -0.01048,  0.64278, -0.14758],
       [-0.27714,  0.29334,  0.2562 ,  0.00374, -0.08475, -0.75424, -0.21903,  0.38399],
       [-0.27145, -0.57526,  0.08172, -0.57334, -0.03944, -0.2762 , -0.10151, -0.41453],
       [-0.17986, -0.0683 ,  0.30335,  0.04763, -0.5    ,  0.44334, -0.63942,  0.11525],
       [-0.457  ,  0.22504,  0.55954,  0.18528,  0.47639,  0.19083,  0.08027, -0.35112],
       [-0.36874, -0.21362,  0.03457, -0.3255 ,  0.17048,  0.3046 ,  0.32217,  0.69686]])

Similarly, we obtain the first 3 rows of $\textbf{V}^{T}_{A}$ from the $\textbf{V}^{T}_{B}$.

In [37]:
Vrt

array([[ 0.468  ,  0.18032,  0.38451,  0.54514,  0.35009,  0.38451,  0.18171],
       [ 0.25032, -0.1055 ,  0.45429, -0.40711, -0.57385,  0.45429, -0.1357 ],
       [-0.49106, -0.31199,  0.10521,  0.6119 , -0.47261,  0.10521,  0.20397]])

In [38]:
Vt

array([[-0.468  , -0.18032, -0.38451, -0.54514, -0.35009, -0.38451, -0.18171],
       [ 0.25032, -0.1055 ,  0.45429, -0.40711, -0.57385,  0.45429, -0.1357 ],
       [ 0.49106,  0.31199, -0.10521, -0.6119 ,  0.47261, -0.10521, -0.20397],
       [-0.52914, -0.27549,  0.51251, -0.2694 ,  0.5346 ,  0.15164,  0.00902],
       [ 0.4194 , -0.86669, -0.06167,  0.06822,  0.1188 , -0.19348, -0.11377],
       [ 0.07815,  0.17015,  0.60343,  0.11666, -0.14369, -0.73389, -0.16723],
       [ 0.12319, -0.05492,  0.06404, -0.26801, -0.06493, -0.1889 ,  0.93058]])

So we can obtain all of the information in $\textbf{A}$ with
# $$ \textbf{A} = \textbf{Q}\textbf{U}_{B} \Sigma_{B}\textbf{V}^{T}_{B} $$

In [39]:
(Q @ Ur @ np.diag(Sigmar) @ Vrt).round().astype(int)

matrix([[ 4,  3,  2, 12,  9,  2,  4],
        [10,  2, 12, 12,  2, 12,  4],
        [ 3,  1,  2,  0,  1,  2,  0],
        [ 7,  2,  6,  3,  2,  6,  1],
        [ 3,  3,  0,  9,  9,  0,  3],
        [ 4,  2,  2,  3,  4,  2,  1],
        [11,  4,  8,  6,  6,  8,  2],
        [ 6,  3,  4,  9,  7,  4,  3]])

In [40]:
A

matrix([[ 4,  3,  2, 12,  9,  2,  4],
        [10,  2, 12, 12,  2, 12,  4],
        [ 3,  1,  2,  0,  1,  2,  0],
        [ 7,  2,  6,  3,  2,  6,  1],
        [ 3,  3,  0,  9,  9,  0,  3],
        [ 4,  2,  2,  3,  4,  2,  1],
        [11,  4,  8,  6,  6,  8,  2],
        [ 6,  3,  4,  9,  7,  4,  3]])

## Now let's take the same approach for the MovieLens data.

In [41]:
A = Rtngs 

In [42]:
A.shape

(611, 9742)

In this case, since we are using a subset version of the MovieLens data in which we are able to store the entire ratings data set in our memory, we can compute the SVD outright to determine how many singular values we want to use to approximate the User Ratings Matrix. However, in most cases, this is not an optimal procedure and is computationally expensive. You can see below, that even with our subset data of the ratings, the time to compute the SVD (at least on my machine) is approximately 9 seconds.

In [43]:
%time U, Sigma, Vt = linalg.svd(A)

Wall time: 9.04 s


Let's suppose we do not have the time or computational resources to compute the SVD outright. How are we going to estimate the reduction parameter as we did in the example? Note that in the example it was clear with the last 4 singular values being zero, that we could reconstruct $\textbf{A}$ with only 3 (r =3). Now that the User Ratings Matrix has 611 singular values, how many do we really need to reconstruct the matrix with an acceptable level of certainty? Here, we introduce the Frobenius Norm.

# $$ ||\textbf{A}||_{F} = \sqrt{tr(\textbf{A}\textbf{A}^{*})} $$

In [61]:
np.linalg.norm(A, ord= 'fro')

1159.6241848116138

So we want to choose an appropriate value such that the frobenius norm of our reduced approximation is close to that of our original User Rating Matrix. For this exercise, we will choose an r value such that we want to achieve 

In [49]:
sum(Sigma[:100])/sum(Sigma)

0.40028613255570544

### Trial 1: Choose r = 100

In [50]:
Ω = np.random.normal(size = (A.shape[1], 100) )
Y = A @ Ω
Q, R = np.linalg.qr(Y, mode = 'reduced') # Q is the Gram-Schmidt process 
B = Q.T @ A

array([[   0.     ,    0.     ,    0.     , ...,    0.     ,    0.     ,    0.     ],
       [-104.9848 ,  -24.09597,  -72.16868, ..., -123.97344, -136.29638,   64.71469],
       [  39.01332,  -14.27205,   26.15262, ...,  -37.75846,   15.36225,   32.7287 ],
       ...,
       [ -11.24709,  -66.78183, -168.29906, ...,  -10.80006,  -88.58394,   81.35586],
       [  -5.91138,  -34.2979 ,    6.1717 , ...,   25.07995,   -8.52633,  -17.93754],
       [ -73.44116,  -91.75189,  -77.29179, ...,   22.11244, -196.84659,  115.04772]])

Now we will compute the SVD of $\textbf{B}$.

In [55]:
%time Ur, Sigmar, Vrt = linalg.svd(B, full_matrices = False)
#sum((Sigmar - Sigma[:len(Sigmar)])**2)/sum(Sigma**2)

Wall time: 202 ms


0.018323990110064293

Note that for $r=100$, computing the SVD is significantly faster that computing it for $\textbf{A}$ outright.

We are going to round the values computed by the reduced SVD approximation to the nearest whole integer, since the user ratings range between 0.5 and 5, by 0.5 increments, we are going to disregard 0.5 ratings in our estimates. 

In [71]:
A_r= (Q @ Ur @ np.diag(Sigmar) @ Vrt).round()

Now by dividing the frobenius norm of the reduced approximation by the frobenius norm of the original, we obtain:

In [75]:
np.linalg.norm(A_r, ord = 'fro') / np.linalg.norm(A, ord = 'fro')

0.7416892657309809

This means that are preserving approximately 75% of the energy for the matrix $\textbf{A}$ with $r=100$. 

### Trial 2: Choose r = 200

In [76]:
Ω = np.random.normal(size = (A.shape[1], 200) )
Y = A @ Ω
Q, R = np.linalg.qr(Y, mode = 'reduced') # Q is the Gram-Schmidt process 
B = Q.T @ A

In [77]:
%time Ur, Sigmar, Vrt = linalg.svd(B, full_matrices = False)
#sum((Sigmar - Sigma[:len(Sigmar)])**2)/sum(Sigma**2)

Wall time: 368 ms


In [78]:
A_r= (Q @ Ur @ np.diag(Sigmar) @ Vrt).round()

In [79]:
np.linalg.norm(A_r, ord = 'fro') / np.linalg.norm(A, ord = 'fro')

0.8518412993069163

Note that we still have significant speed and with $r=200$, we are preserving approximately 85% of the energy for the matrix $\textbf{A}$

### Trial 3: Choose r = 300

In [84]:
Ω = np.random.normal(size = (A.shape[1], 300) )
Y = A @ Ω
Q, R = np.linalg.qr(Y, mode = 'reduced') # Q is the Gram-Schmidt process 
B = Q.T @ A

In [85]:
%time Ur, Sigmar, Vrt = linalg.svd(B, full_matrices = False)
#sum((Sigmar - Sigma[:len(Sigmar)])**2)/sum(Sigma**2)

Wall time: 511 ms


In [86]:
A_r= (Q @ Ur @ np.diag(Sigmar) @ Vrt).round()

In [87]:
np.linalg.norm(A_r, ord = 'fro') / np.linalg.norm(A, ord = 'fro')

0.9139647693022843

Note that we still have significant speed and with $r=300$, we are preserving approximately 91% of the energy for the matrix $\textbf{A}$.

### Conclusion
With approximately half of the singular values from the SVD, we are able to reconstruct the original User Rating Matrix up to about 91%. Further trials can approximate an appropriate choice in our reduction value that can help us do these types of computations with larger data sets using a similar approach. The SVD is incredible tool that can be further used in predicting movie ratings and recommending movies to users as well. 